# Library

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# [Load] 230206_RH_4등급DB세분류_인증번호변경 건_v1.2

In [2]:
fold = 'D:/data/big2/4등급_배인번호_재확인'

In [10]:
file_name = '4등급DB세분류_인증번호변경 건_2302091522.csv'
df = pd.read_csv(os.path.join(fold, file_name), encoding='cp949', low_memory=False)

In [11]:
# 14.5+ MB
# low_memory=False, 14.5+ MB
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61417 entries, 0 to 61416
Data columns (total 31 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   등급(현재)     61417 non-null  object 
 1   말소(현재)     61417 non-null  object 
 2   인증번호(현재)   61417 non-null  object 
 3   인증번호(변경전)  2368 non-null   object 
 4   인증번호(변경후)  61417 non-null  object 
 5   차대번호       61417 non-null  object 
 6   차량번호       61417 non-null  object 
 7   제원관리번호     61417 non-null  object 
 8   차종         61417 non-null  object 
 9   용도         61417 non-null  object 
 10  최초등록일      61417 non-null  int64  
 11  연식         61417 non-null  int64  
 12  제작일자       25286 non-null  float64
 13  검사유효일      61415 non-null  float64
 14  등급         61417 non-null  object 
 15  차명         61417 non-null  object 
 16  차종분류       61417 non-null  object 
 17  차종유형       61417 non-null  object 
 18  자동차형식      61410 non-null  object 
 19  제작사명       60375 non-null  object 
 20  연료    

In [12]:
df.iloc[:5, 22]

0     2960
1     2920
2     2960
3    15455
4    15455
Name: 총중량, dtype: object

# Check data

## 기초

In [13]:
df.isnull().sum()

등급(현재)           0
말소(현재)           0
인증번호(현재)         0
인증번호(변경전)    59049
인증번호(변경후)        0
차대번호             0
차량번호             0
제원관리번호           0
차종               0
용도               0
최초등록일            0
연식               0
제작일자         36131
검사유효일            2
등급               0
차명               0
차종분류             0
차종유형             0
자동차형식            7
제작사명          1042
연료               0
엔진형식           266
총중량              0
적재중량            22
엔진출력            88
배기량              0
법정동코드            0
시도명              0
시군구명             0
소유자구분            0
말소여부             0
dtype: int64

In [21]:
df['등급(현재)'].value_counts(dropna=False)

4     60714
5       679
3        12
T4       12
Name: 등급(현재), dtype: int64

In [22]:
df['등급'].value_counts(dropna=False)

4     60714
5       678
T4       13
3        12
Name: 등급, dtype: int64

In [15]:
df['말소(현재)'].value_counts(dropna=False)

N    59353
Y     2064
Name: 말소(현재), dtype: int64

In [16]:
df['말소여부'].value_counts(dropna=False)

N     61416
주민        1
Name: 말소여부, dtype: int64

### 말소여부 이상치
- '주민' 값 존재
    - '제작사명' 값부터 하나씩 오른쪽 열로 밀림

In [17]:
df[df['말소여부'] == '주민']

,등급(현재),말소(현재),인증번호(현재),인증번호(변경전),인증번호(변경후),차대번호,차량번호,제원관리번호,차종,용도,...,엔진형식,총중량,적재중량,엔진출력,배기량,법정동코드,시도명,시군구명,소유자구분,말소여부
14783,4,N,7MY-HD-14-68,NaN,7MY-HD-14-68,KN94J4MCDCHZ00003,인천83배1254,A0810006211533112,화물,영업용,...,DS,D4CB,3140.0,1000.0,133,2497,2818510400,인천광역시,연수구,주민


In [20]:
df.loc[df['말소여부'] == '주민', '자동차형식':]

,자동차형식,제작사명,연료,엔진형식,총중량,적재중량,엔진출력,배기량,법정동코드,시도명,시군구명,소유자구분,말소여부
14783,HR-3NDSJKA-E521,NaN,현대자동차(주),DS,D4CB,3140.0,1000.0,133,2497,2818510400,인천광역시,연수구,주민


In [24]:
df[df['말소(현재)'] != df['말소여부']].shape

(2065, 31)

In [25]:
df[df['말소(현재)'] != df['말소여부']]

,등급(현재),말소(현재),인증번호(현재),인증번호(변경전),인증번호(변경후),차대번호,차량번호,제원관리번호,차종,용도,...,엔진형식,총중량,적재중량,엔진출력,배기량,법정동코드,시도명,시군구명,소유자구분,말소여부
0,4,Y,9MY-KM-14-41,7MY-KM-14-64,9MY-KM-14-41,KNHMB76438S237203,74마6376,A0110003600592207,승합,자가용,...,J3,2960,0.0,192.0,2902,2915510900,광주광역시,남구,주민,N
9,4,Y,7MY-DB-23-43,7MY-DB-23-43,7MY-DB-24-43,KL5UF65RD8P000124,75구9173,A0210003100712307,승합,자가용,...,DV11,15455,0.0,430.0,10964,4146110800,경기도,용인시 처인구,주민,N
11,4,Y,7MY-DB-23-43,7MY-DB-23-43,7MY-DB-24-43,KL5UF65RD8P000130,76거4382,A0210003100712307,승합,자가용,...,DV11,15455,0.0,430.0,10964,4480025026,충청남도,홍성군,주민,N
16,4,Y,7MY-DB-24-43,7MY-DB-23-43,7MY-DB-24-43,KL5UF65RD8P000177,72노0245,A0210003100712307,승합,자가용,...,DV11,15455,0.0,430.0,10964,4311312000,충청북도,청주시 흥덕구,법인,N
20,4,Y,7MY-DB-23-43,7MY-DB-23-43,7MY-DB-24-43,KL5UF65RD8P000220,74버5529,A0210003100712307,승합,자가용,...,DV11,15455,0.0,430.0,10964,2818510400,인천광역시,연수구,주민,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61275,4,Y,5MY-PA-14-17,NaN,5MY-PA-14-17,YV1RS714062538408,10오6614,9.20E+14,승용,자가용,...,D5244T,1940,0.0,185.0,2401,1162010200,서울특별시,관악구,주민,N
61276,4,Y,5MY-PA-14-17,NaN,5MY-PA-14-17,YV1RS714062538480,02로0919,9.20E+14,승용,자가용,...,D5244T,1940,0.0,185.0,2401,4127310100,경기도,안산시 단원구,주민,N
61279,4,Y,5MY-PA-14-17,NaN,5MY-PA-14-17,YV1RS714062545028,41무1909,9.20E+14,승용,자가용,...,D5244T,1940,0.0,185.0,2401,1144010700,서울특별시,마포구,법인,N
61285,4,Y,5MY-PA-14-17,NaN,5MY-PA-14-17,YV1RS714062552057,41보6233,9.20E+14,승용,자가용,...,D5244T,1940,0.0,185.0,2401,2714010200,대구광역시,동구,주민,N
